## Simulação Molecular (Dinâmica Molecular)

Anotações:
* Modelo de Maxwell-Boltzmann
* Conservação de momento e energia
* Simulação em uma caixa, inicialmente bidimensional
* Distribuição estatística em função da temperatura (alteração de velocidade)

Leituras:
https://introcs.cs.princeton.edu/java/assignments/collisions.html
McQuirre, últimos 4 capítulos - amauri vai passar especificamente

Para aula 08:
* código que descreva uma partícula em movimento (ou pelo menos uma ideia)

In [1]:
import numpy as np
import tkinter as tk
import math
import random
import pygame

pygame 2.5.0 (SDL 2.28.0, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#constantes
#temperatura = 25 C #devem obedecer a distribuição de maxwell-boltzmann
tamanho_caixa = [800,600] #[0] == largura, [1] == altura
FPS = 120
dt = 1/FPS

In [3]:
class Particle:
    def __init__(self, massa, posicao, raio, velocidade, cor):
        self.massa = massa
        self.raio = raio
        self.velocidade = velocidade #vetor
        self.posicao = posicao #vetor
        self.aceleracao = [1,1] #aceleracao constante, aqui em apenas 1D
        self.cor = cor
        
    def draw(self, screen):
        pygame.draw.circle(screen, self.cor, (int(self.posicao[0]), int(self.posicao[1])), self.raio)

In [4]:
def update_position(particula,dt):
    nova_velocidade_x = particula.velocidade[0] + particula.aceleracao[0] * dt #atualiza velocidade em x
    nova_velocidade_y = particula.velocidade[1] + particula.aceleracao[1] * dt #atualiza velocidade em y
    
    nova_posicao_x = particula.posicao[0] + particula.velocidade[0] * dt
    nova_posicao_y = particula.posicao[0] + particula.velocidade[1] * dt
    
    return nova_posicao_x,nova_posicao_y,nova_velocidade_x,nova_velocidade_y

In [5]:
def colisao_caixa(particula):
    if tamanho_caixa[0] - particula.raio > 0 or tamanho_caixa[0] + particula.raio > tamanho_caixa:
        inverte_x = -(particula.velocidade[0])
        
        return [inverte_x,particula.velocidade[1]]
        
    if tamanho_caixa[1] - particula.raio > 0 or tamanho_caixa[1] + particula.raio > tamanho_caixa:
        inverte_y = -(particula.velocidade[1])
        
        return [particula.velocidade[0],inverte_y]
    
    if tamanho_caixa[0] - particula.raio < 0 or tamanho_caixa[0] + particula.raio < tamanho_caixa:
        nao_inverte_x = particula.velocidade[0]
        
        return [nao_inverte_x,particula.velocidade[1]]
    
    if tamanho_caixa[1] - particula.raio < 0 or tamanho_caixa[1] + particula.raio < tamanho_caixa:
        nao_inverte_y = particula.velocidade[1]
        
        return [particula.velocidade[0],nao_inverte_y]


In [6]:
def colisao_particula(particula_1,particula_2):
    
    soma_raios = particula_1.raio + particula_2.raio #soma dos raios
    
    distancia_euclidiana = math.sqrt((particula_2.posicao[0] - particula_1.posicao[0])**2 + (particula_2.posicao[1] - particula_1.posicao[1])**2)
    if distancia_euclidiana < soma_raios:
    
        lambda_x = particula_1.posicao[0] - particula_2.posicao[0] #numero
        lambda_y = particula_1.posicao[1] - particula_2.posicao[1] #numero
    
        lambda_vel_x = particula_1.velocidade[0] - particula_2.posicao[0] #numero
        lambda_vel_y = particula_1.velocidade[1] - particula_2.posicao[1] #numero
    
        vet_pos = np.array([lambda_x, lambda_y]) #vetor
        vet_vel = np.array([lambda_vel_x, lambda_vel_y]) #vetor
        #print("done - ",vet_vel*vet_pos)
    
        impulso = (2*particula_1.massa * particula_2.massa * (np.inner(vet_vel,vet_pos))) / (soma_raios * (particula_1.massa + particula_2.massa)) #impulso inelastico
    
        impulso_x = (impulso*lambda_x)/soma_raios #impulso no eixo x
        impulso_y = (impulso*lambda_y)/soma_raios #impulso no eito y
    
    
        vr_p1_x = particula_1.velocidade[0] + (impulso_x/particula_1.massa) #velocidade resultante particula 1 no eixo x
        vr_p1_y = particula_1.velocidade[1] + (impulso_y/particula_1.massa)
    
        vr_p2_x = particula_2.velocidade[0] - (impulso_x/particula_2.massa) #velocidade resultante particula 2 no eixo x
        vr_p2_y = particula_2.velocidade[1] - (impulso_y/particula_2.massa)
        
        
        return vr_p1_x,vr_p1_y,vr_p2_x,vr_p2_y
    
    else:
        vr_p1_x = particula_1.velocidade[0]
        vr_p1_y = particula_1.velocidade[1]
        
        vr_p2_x = particula_2.velocidade[0]
        vr_p2_y = particula_2.velocidade[1]
        
        return vr_p1_x,vr_p1_y,vr_p2_x,vr_p2_y
        

In [7]:
def caracterizar_particulas(n_particulas,tamanho_caixa):
    particulas = []
    for _ in range(n_particulas):
        massa = random.uniform(1, 5)
        velocidade = [random.uniform(-5, 5), random.uniform(-5, 5)]
        raio = 5
        posicao = [random.randint(raio,tamanho_caixa[0]-raio), random.randint(raio,tamanho_caixa[1]-raio)]
        cor = "#%02x%02x%02x" % (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        particulas.append(Particle(massa, posicao, raio, velocidade, cor))
    return particulas

In [8]:
#GUI com pygame
WHITE = (255, 255, 255)
screen = pygame.display.set_mode((tamanho_caixa[0], tamanho_caixa[1]))
pygame.display.set_caption("Particle Collision Simulation")
clock = pygame.time.Clock()
particulas = caracterizar_particulas(2,tamanho_caixa)

# Main loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    for p_1 in range (0,len(particulas)):
        for p_2 in range(p_1 + 1,len(particulas)):
            vr_p1_x,vr_p1_y,vr_p2_x,vr_p2_y = colisao_particula(particulas[p_1],particulas[p_2]) #checa todos os pares para colisao entre particulas
            particulas[p_1].velocidade = [vr_p1_x,vr_p1_y]
            particulas[p_2].velocidade = [vr_p1_x,vr_p1_y] 
        

    for p in particulas: 
        box_col = colisao_caixa(p) #checa todas, de um em u
        p.velocidade = box_col
        
        
        nova_posicao_x,nova_posicao_y,nova_velocidade_x,nova_velocidade_y = update_position(p,dt)
        p.posicao = [nova_posicao_x,nova_posicao_y]
        p.velocidade = [nova_velocidade_x,nova_velocidade_y]
    
    # Clear the screen
    screen.fill(WHITE)
    
    # Draw particles
    for p in particulas:
        p.draw(screen)
    
    # Update the display
    pygame.display.flip()
    clock.tick(FPS)

# Clean up
pygame.quit()